In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='7nIjyeNwQGotwVmv7JJ7EMyPlFP6ItolvBFfLN8SslUr',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'forestfire-donotdelete-pr-6ottikia2kffoz'
object_key = 'archive.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [2]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [3]:
ls

Dataset/  forest1.h5


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
import keras
from keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 rotation_range=180,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)


In [7]:
x_train = train_datagen.flow_from_directory(r'Dataset/Dataset/train_set',
                                        target_size =(128,128),
                                        batch_size = 32,
                                        class_mode ='binary')

Found 436 images belonging to 2 classes.


In [8]:
x_test = train_datagen.flow_from_directory(r'Dataset/Dataset/test_set',
                                        target_size =(128,128),
                                        batch_size = 32,
                                        class_mode ='binary')

Found 121 images belonging to 2 classes.


model build

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [10]:
model=Sequential()

In [11]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [12]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
model.add(Flatten())

In [14]:
model.add(Dense(150,activation='relu'))

In [15]:
model.add(Dense(1,activation='sigmoid'))

In [16]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

In [17]:
model.fit_generator(x_train,steps_per_epoch=14,epochs=10,validation_data=x_test,validation_steps=4)

Epoch 1/10
14/14 [==============================] - 47s 3s/step - loss: 3.1758 - accuracy: 0.6101 - val_loss: 0.2490 - val_accuracy: 0.9008
Epoch 2/10
14/14 [==============================] - 45s 3s/step - loss: 0.7200 - accuracy: 0.8188 - val_loss: 0.1527 - val_accuracy: 0.9339
Epoch 3/10
14/14 [==============================] - 47s 3s/step - loss: 0.3635 - accuracy: 0.8463 - val_loss: 0.1334 - val_accuracy: 0.9421
Epoch 4/10
14/14 [==============================] - 46s 3s/step - loss: 0.2540 - accuracy: 0.8922 - val_loss: 0.1293 - val_accuracy: 0.9421
Epoch 5/10
14/14 [==============================] - 47s 3s/step - loss: 0.1920 - accuracy: 0.9197 - val_loss: 0.1021 - val_accuracy: 0.9669
Epoch 6/10
14/14 [==============================] - 47s 3s/step - loss: 0.1898 - accuracy: 0.9266 - val_loss: 0.0893 - val_accuracy: 0.9752
Epoch 7/10
14/14 [==============================] - 48s 3s/step - loss: 0.1624 - accuracy: 0.9404 - val_loss: 0.0894 - val_accuracy: 0.9835
Epoch 8/10
14/14 [==

In [18]:
model.save("forest12.h5")

In [19]:
!tar  -zcvf firedetection.tgz 'forest12.h5'

forest12.h5


In [20]:
!pip install ibm_watson_machine_learning

In [21]:
from ibm_watson_machine_learning import APIClient

In [22]:
wml_credentials={
    "url":"https://eu-de.ml.cloud.ibm.com",
    "apikey":"40od7r0fKED2ImiM73SxulP-Esy3j1WtoRQM2sgwBUsT"
}

In [23]:
client = APIClient(wml_credentials)

In [24]:
client

In [25]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------------------  ------------------------
ID                                    NAME                   CREATED
d409bee6-b69f-4bad-811b-b0a65287bd93  forest fire detection  2022-11-17T05:50:45.177Z
------------------------------------  ---------------------  ------------------------


In [26]:
space_id = "d409bee6-b69f-4bad-811b-b0a65287bd93"

In [27]:
client.set.default_space(space_id)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [29]:
software_space_uid = client.software_specifications.get_id_by_name("tensorflow_rt22.1-py3.9")

In [30]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [31]:
model_details = client.repository.store_model(model='firedetection.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"cnn model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [32]:
model_id = client.repository.get_model_id(model_details)

In [33]:
model_id

'30a2cdc4-ecdf-48cc-bb0b-96769fcdb6a5'